## Задание

Необходимо сделать прототип RAG системы на документах из папки data которая будет отвечать на следующие вопросы:
```
1) В каких зонах по весу снежного покрова находятся Херсон и Мелитополь?
2) Какие регионы Российской Федерации имеют высотный коэффициент  k_h , превышающий 2?
3) Выведи рекомендуемые варианты конструктивного решения заземлителей для стержневых молниеприемников.
4) Что означает аббревиатура 'ТС'?
5) Что должна содержать Пояснительная записка в графической части?
6) Сколько разделов должна содержать проектная документация согласно 87ому постановлению?
7) Какая максимальная скорость движения подземных машин в выработках?
8) Какая максимальная температура допускается в горных выработках?
9) Какие допустимые значения по отклонению геометрических параметров сечения горных выработок?
10) В каком пункте указана минимальная толщина защитного слоя бетона для арматуры при креплении стволов монолитной бетонной крепью?
```
Ответы должны быть короткими а также содержать ссылку на документы, на основании которых был составлен ответ.

Что интересует в решениии:
1) Предобработка данных, чанкинг, выбор индекса
2) RAG пайплайн, выбранный фреймворк для построения системы

Сдача решения:
1) Необходимо представить свое решение в формате презентации где детально рассказать о том что попробовали, что получилось, показать код решения а также ответы системы. Представить свои размышления по улучшению системы, где "узкое горлышко" и тд.
2) Быть готовым ответить на вопрос по решению.

## Презентация логики построения RAG-системы: структура и обоснование

### 1. Предобработка данных и чанкинг

Предобработка данных и чанкинг:
* Использование структуры документа (центрированные заголовки, цифровые подразделы) для автоматической извлечении иерархии через `.find()` и регулярные выражения
* Группировка буквенных подпунктов в родительские чанки для сохранения контекста
* Представить документ в JSON-структуре
* Overlapping chunks

Почему так:
* Контекстная целостность: чанки отражают логическую структуру документа, что улучшает релевантность поиска
* Простота реализации: минимизация зависимости от тяжелых библиотек и вычислительных мощностей
* JSON-структура документа позволяет легко фильтровать и индексировать данные

Слабые места:
* Хрупкость к формату: решение зависит от строго оформления документов
* Ручные правила: требует точного знания структуры всех вводных документов

Потенциальные улучшения:
* Добавить парсинг стилей, если документ сам нам подсказывает своим оформлением на структурную единицу документа
* Использовать LLM для точного выуживания названий разделов и подразделов

### 2. RAG-пайплайн и выбор фреймворка

* В качестве основного фреймворка был выбран LangChain: интеграция с open-source моделями посредством API, PydanticOutputParser - строгая типизация и валидация выходный данных, упрощённое взаимодействие с моделями
* LLM как роутер: модель анализирует вопрос, выбирает документы и разделы через Pydantic-объект `queries`
* Delegation Agent + BM25 Retriever: поиск по предобработанным чанкам с возвратом контекста в LLM для финального ответа

Почему так:
* Гибкость запросов: LLM-роутинг позволяет обрабатывать сложные вопросы с фильтрацией по разделам
* Контроль контекста: Pydantic гарантирует структурированный ввод / вывод, снижая риск галлюцинаций
* Легковесность: BM25 работает быстро на небольших корпусах и не требует GPU

Слабые места:
* BM25 объективно уступает ретриверам на эмбеддингах в задачах семантического поиска Низкая точность на синонимах и сложных запросах
* Многократные вызовы модели (роутинг + финальный ответ) увеличивают время ответа

Потенциальные улучшения:
* Реализовать гибридный поиск
* Кэшировать результаты роутинга для повторяющихся запросов
* Векторизация чанков для семантического поиска

### 3. Заключение

* Решение подходит для строго форматированных документов (юридические тексты, ГОСТы, СП)
* Минималистичный стек технологий снижает порог входа и упрощает поддержку

## Практическая реализация решения

In [1]:
%%capture
!pip install langchain langchain-community langchain-openai langchain-huggingface huggingface_hub python-docx

In [ ]:
import re

# /content/Контрольные_Вопросы.md
file_path = '/content/Контрольные_Вопросы.md'

# Открываем файл и читаем его содержимое
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Используем регулярное выражение для извлечения вопросов
questions = re.findall(r'\d+\)\s*(.*?\?)', content)

# Выводим список вопросов
print(questions)

['В каких зонах по весу снежного покрова находятся Херсон и Мелитополь?', 'Какие регионы Российской Федерации имеют высотный коэффициент  k_h , превышающий 2?', "Что означает аббревиатура 'ТС'?", 'Что должна содержать Пояснительная записка в графической части?', 'Сколько разделов должна содержать проектная документация согласно 87ому постановлению?', 'Какая максимальная скорость движения подземных машин в выработках?', 'Какая максимальная температура допускается в горных выработках?', 'Какие допустимые значения по отклонению геометрических параметров сечения горных выработок?', 'В каком пункте указана минимальная толщина защитного слоя бетона для арматуры при креплении стволов монолитной бетонной крепью?']


In [2]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.retrievers import BM25Retriever
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH

import re
import os
import json

# Путь к документу
doc_path = '/content/О СОСТАВЕ РАЗДЕЛОВ ПРОЕКТНОЙ ДОКУМЕНТАЦИИ И ТРЕБОВАНИЯХ К ИХ СОДЕРЖАНИЮ.docx'

def form_document(doc_path: str) -> str:
    """
    Открывает документ и извлекает его текст
    """
    doc = Document(doc_path)
    # Собираем текст из всех абзацев, разделяя их табуляцией
    document = '\n'.join([paragraph.text for paragraph in doc.paragraphs if paragraph.text.strip()])
    return document

def extract_centered_paragraphs(doc_path: str) -> list:
    """
    Извлекает все центрированные элементы из документа

    :param doc_path: Путь к файлу
    :return: Список центрированных элементов
    """
    doc = Document(doc_path)
    centered_paragraphs = []

    for paragraph in doc.paragraphs:
        if paragraph.paragraph_format.alignment == WD_ALIGN_PARAGRAPH.CENTER:
            centered_paragraphs.append(paragraph.text)

    return centered_paragraphs

def extract_sections(document: str, section_titles: list) -> dict:
    """
    Извлекает разделы документа на основе списка заголовков

    :param document: Полный текст документа
    :param section_titles: Список заголовков разделов
    :return: Словарь, где ключ - заголовок раздела, значение - текст раздела
    """
    sections = {}
    for idx, title in enumerate(section_titles):
        # Находим начало текущего раздела
        start = document.find(title)

        if start == -1:
            continue
        # Определяем конец текущего раздела как начало следующего заголовка
        next_start = document.find(section_titles[idx + 1]) if idx + 1 < len(section_titles) else len(document)
        end = next_start
        # Извлекаем текст раздела
        sections[title] = document[start:end].strip()
    return sections

def extract_sub_sections(document: str) -> list:
    """
    Извлекает подразделы, определяемые как строки, начинающиеся с цифры и точки

    :param document: Полный текст документа
    :return: Список найденных подразделов
    """
    sub_sections = []
    lines = document.split('\n')
    for line in lines:
        stripped_line = line.strip()
        # Проверяем, начинается ли строка с цифры и точки
        if re.match(r'^\d+\.', stripped_line):
            sub_sections.append(stripped_line)
    return sub_sections

def extract_section_title(text):
    """
    Извлекает название раздела из строки, предполагая, что оно заключено в кавычки

    :param text: Строка, в которой нужно найти название раздела
    :return: Название раздела, если оно найдено, иначе None
    """
    # Регулярное выражение для поиска названия раздела
    pattern = r'Раздел \d+ "([^"]+)"'
    match = re.search(pattern, text)

    if match:
        # Если совпадение найдено, извлекаем название раздела
        section_title = match.group(1)
        return section_title
    else:
        # Если совпадение не найдено, возвращаем None
        return ''

def extract_preprocessed_sections(document: str, section_titles: list) -> dict:
    """
    Извлекает разделы документа на основе списка заголовков

    :param document: Полный текст документа
    :param section_titles: Список заголовков разделов
    :return: Словарь, где ключ - заголовок раздела, значение - текст раздела
    """
    sections = {}
    for idx, title in enumerate(section_titles):
        # Находим начало текущего раздела
        start = document.find(title)

        if start == -1:
            continue
        # Определяем конец текущего раздела как начало следующего заголовка
        next_start = document.find(section_titles[idx + 1]) if idx + 1 < len(section_titles) else len(document)
        end = next_start
        # Извлекаем текст раздела
        if extract_section_title(title) != '':
            sections[extract_section_title(title)] = document[start:end].strip()
    return sections

In [ ]:
text = """
 '6. Раздел 5 "Проект организации работ по сносу (демонтажу) линейного объекта", включаемый в состав проектной документации при необходимости сноса (демонтажа) линейного объекта или части линейного объекта, в текстовой части содержит документы и сведения, указанные в подпунктах "ф.1" и "ш" пункта 23 Положения, а также перечень проектных решений по устройству временных сетей инженерно-технического обеспечения на период строительства линейного объекта (при наличии объектов, подлежащих сносу (демонтажу), попадающих в зону строительства сетей газораспределения и (или) газопотребления).'
"""

extract_section_title(text)

'Проект организации работ по сносу (демонтажу) линейного объекта'

In [3]:
# Функция для создания JSON-документа из строки
def create_json_document(doc_path: str, output_file: str = 'О_Составе_Разделов_ПД.json'):
    """
    Основная функция для обработки документа и сохранения результата в JSON-файл.

    :param doc_path: Путь к документу для обработки.
    :param output_file: Имя выходного JSON-файла (по умолчанию 'О_Составе_Разделов_ПД.json').
    """
    # Извлекаем текст документа
    document = form_document(doc_path)  # String: Полный текст документа

    # Извлекаем центрированные параграфы (например, заголовки)
    centered_paragraphs = extract_centered_paragraphs(doc_path)  # List: Список центрированных параграфов

    # Извлекаем подразделы документа
    sub_sections = extract_sub_sections(document)  # List: Список подразделов

    # Извлекаем разделы документа на основе центрированных параграфов
    document_dict = extract_sections(document, centered_paragraphs)  # Dict: Словарь с разделами документа

    # Обрабатываем каждый раздел для извлечения подразделов
    final_document_dict = {}

    for key, value in document_dict.items():
        # Для каждого раздела извлекаем подразделы
        final_document_dict[key] = extract_preprocessed_sections(value, sub_sections)  # Dict: Вложенный словарь с подразделами

    # Сохраняем результат в JSON-файл
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_document_dict, f, indent=4, ensure_ascii=False)
    print(f'Документ успешно сохранён в файл: {output_file}')

create_json_document(doc_path)

Документ успешно сохранён в файл: О_Составе_Разделов_ПД.json


In [4]:
with open('О_Составе_Разделов_ПД.json', 'r', encoding='utf-8') as f:
    project_structure = json.load(f)

# Сбор длин всех элементов списков на последнем уровне вложенности
char_counts = []
for outer_key, inner_dict in project_structure.items():
    for inner_key, value_list in inner_dict.items():
        char_counts.append(len(value_list))

# Вычисление среднего значения
average = sum(char_counts) / len(char_counts)
print(f"Среднее число символов одного элемента списка: {average:.2f}")

Среднее число символов одного элемента списка: 3249.02


In [7]:
from langchain_huggingface.llms.huggingface_endpoint import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    task='text-generation',
    model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
    temperature=0.1,
    huggingfacehub_api_token='token'
)

In [8]:
# Pydantic-модель для роутинга
class queries(BaseModel):
    sections: list[str]
    subsections: list[str]
    chain_of_thought: str

# Создание словаря, где ключи — это ключи верхнего уровня, а значения — ключи первой вложенности
structure = {outer_key: list(inner_dict.keys()) for outer_key, inner_dict in project_structure.items()}
document_structure = str(structure)

parser = PydanticOutputParser(pydantic_object = queries)
format_instructions = parser.get_format_instructions

question = """
Что написано в разделе охраны окружающей среды для атомных станций?
"""

system_prompt_delegation = f"""
Ты будешь получать запросы, ответы на который есть в документе.
У вас тебя есть к специализированным агентам, которые могут извлекать данные из документа.
Документ имеет следующую структуру: {document_structure}

Чтобы делегировать задачи этим агентам, следуй следующим рекомендациям:

1. Определите раздела:
    - Укажи точные названия разделов документа, в которых потенциально можно будет найти информация для ответа и перечисли их в разделе 'sections'. Убедитесь, что названия разделов точно соответствуют тем, что указаны в {list(structure.keys())}.

2. Определения подразделов:
    - Для каждой идентифицированного раздела укажи также и подразделы в 'subsections', в которых специализированные агенты смогут найти нужную информацию.
    - Перечисли эти запросы в разделе 'queries'.

3. Цепочка рассуждений: Предоставь 'chain_of_thought', объясняя, почему именно эти разделы и подразделы ты выбрал для достижения результата.

{format_instructions}

Запрос для поиска: {question}

Верни ответ в JSON формате!
"""

answer = llm.invoke(system_prompt_delegation)
parsed_answer = parser.parse(answer).dict()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

In [9]:
parsed_answer

{'sections': ['ОСОБЕННОСТИ СОСТАВА РАЗДЕЛОВ ПРОЕКТНОЙ ДОКУМЕНТАЦИИ ДЛЯ АТОМНЫХ СТАНЦИЙ И ТРЕБОВАНИЙ К ИХ СОДЕРЖАНИЮ'],
 'subsections': ['Мероприятия по охране окружающей среды'],
 'chain_of_thought': 'Для ответа на запрос о том, что написано в разделе охраны окружающей среды для атомных станций, мы должны обратиться к разделу, посвященному особенностям составления проектной документации для атомных станций. В этом разделе содержится информация о требованиях к содержанию разделов проектной документации, в том числе и о мероприятиях по охране окружающей среды.'}

In [12]:
project_structure['ОСОБЕННОСТИ СОСТАВА РАЗДЕЛОВ ПРОЕКТНОЙ ДОКУМЕНТАЦИИ ДЛЯ АТОМНЫХ СТАНЦИЙ И ТРЕБОВАНИЙ К ИХ СОДЕРЖАНИЮ']['Мероприятия по охране окружающей среды']

'13. Раздел 8 "Мероприятия по охране окружающей среды" дополнительно содержит:\nв текстовой части\nа) сведения по оценке радиационного воздействия на население и окружающую среду при ситуациях, учитываемых проектом, в том числе вызванных техногенными и природными явлениями, прогноз миграции радионуклидов в поверхностных и подземных водах и прогноз их накопления в донных отложениях;\nб) результаты расчетов приземных концентраций загрязняющих веществ, в том числе радиоактивных, анализ и предложения по нормативам допустимых выбросов;\nв) сведения о характеристиках образующихся радиоактивных отходов (агрегатное состояние газообразных радиоактивных отходов, жидких радиоактивных отходов, твердых радиоактивных отходов, удельная активность, годовое количество (масса), радионуклидный состав, активность по отдельным радионуклидам, классификация по критериям отнесения радиоактивных отходов к особым и удаляемым, по классам удаляемых радиоактивных отходов, сведения о порядке обращения, меры по пред